# **Objetivo del Cuaderno Jupyter**
- El objetivo de los cuadernos `01_normalizando_internet.ipynb` y `02_normalizando_otros.ipynb` es Normalizar la información inicial al esquema mostrado en la Imagen de abajo.
- En este cuaderno en particular obtendremos las tablas relacionadas con el archivo `internet.xlsx`<br>
<br>
<img src='assets/EDR.png'>
<br>

#### **A tomar en cuenta:**
- Basaremos el estudio de la data a nivel de Provincias, Localidades y Total País
- Las tablas `provincias`, `partidos` y `localidades`, aunque no era necesario normalizarlas para el correspondiente EDA y Visualización del Proyecto, se trabajó, para dejarle la estructura de datos armada a la empresa, donde toca ordenar toda la Data (funcional), tanto para este estudio, como para posteriores estudios propios o de cualquier persona de la empresa, quedando dichas tablas listas para ser enlaces entre Futuras Hipotéticas tablas que involucren todas las localidades a detalle. 

#### **Importante**
- Aunque los cuadernos `01_normalizando_internet.ipynb` y `02_normalizando_otros.ipynb` no son se encargan de hacer `EDA`, si se encargarán del `tratamiento de los datos NULOS`, ya que estamos Normalizando tablas que van a ser vaciadas en un Motor SQL para la Empresa y esa data debe ser Guardada ya saneada de Nulos. 
#### **Puntos Extras**
- Probaré el vaciado de las tablas en una base de Datos Mysql.
- Sí desea disponer de las tablas en Mysql en su ordenador, se necesita crear la estructura de Tablas y relaciones primero.  En el archivo `db/tables.py` dispone de toda la estrucutura apoyada en el `ORM SqlAlchemy` y con sólo correr el script `db/create_tables.py` para obtenerla. 
- Sí no posee Mysql en su ordenador puede seguir las intrucciones del archivo `docker/pasos_en_la_terminal.txt` para que cree un contenedor con una imagen de MySql y luego ejecute el Script para crear la estructura.
<br>


Importando Librerias

In [1]:
import polars as pl
import duckdb

Importando Utilidades propias y variables

In [2]:
from utils.utilidades import reemplazar, correcion,obtener_coordenadas

Importando Conexion a Base de Datos

In [111]:
from db.connection import engine_sqlalchemy

***
## **Creando la Tabla `provincias`**

In [ ]:
# Leo un Dataset donde estén las 24 Provincias (23 + Ciudad Autonóma de Buenos Aires)
acc_vel_loc_sinrangos = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Acc_vel_loc_sinrangos')

# Paso las Provincias a una Lista
provincias_list = acc_vel_loc_sinrangos['Provincia'].unique().to_list()

# Si ordeno las Provincias Alfabéticamente esos son los código que le corresponden
provincias_list.sort()
codigos = [6,2,10,22,26,14,18,30,34,38,42,46,50,54,58,62,66,70,74,78,82,86,94,90]

provincias_dict = {
    'id_provincia': codigos,
    'provincia': provincias_list
}

# Creo el Dataframe y paso a Letra Capital el campo Provincia
provincias = pl.DataFrame(data=provincias_dict)
provincias = provincias.with_columns(pl.col('provincia').str.to_titlecase().alias('provincia'))
provincias = provincias.sort('id_provincia')

# Normalizo los nombres de las Provincias en el archivo utilidades.py estan las variables que utilizo
for i in range(len(reemplazar)):
    provincias = provincias.with_columns(
        pl.when(pl.col('provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('provincia'))
          .alias('provincia')
    )

provincias.head(1)

Pensando a futuro, es probable que para objetos de visualizaciones necesitemos las coordenadas de las Provincias de Argentina. La manera mas ordenada de tener esa información pienso es en esta tabla

In [5]:
# La funcion obtener_coordenadas se encuentra en el archivo utilidades.py 
coordenadas = obtener_coordenadas(provincias=provincias['provincia'].to_list())

In [7]:
# Creo los campos dentro del dataframe
provincias = provincias.with_columns([
    pl.lit(0.0).alias('longitud'),
    pl.lit(0.0).alias('latitud')
])

Añado las coordenadas obtenidas al dataframe

In [ ]:
for diccionario in coordenadas:
    provincias = provincias.with_columns([
        pl.when(pl.col('provincia') == diccionario['provincia'])
          .then(pl.lit(diccionario['longitud']))
          .otherwise(pl.col('longitud'))
          .alias('longitud'),
        
        pl.when(pl.col('provincia') == diccionario['provincia'])
          .then(pl.lit(diccionario['latitud']))
          .otherwise(pl.col('latitud'))
          .alias('latitud')

    ])
provincias.head()

In [10]:
del provincias_dict
del provincias_list

***
## **Creando la Tabla `internet_pais`**
Para la creacion de esta tabla necesito los siguientes datasets
- Totales VMD
- Penetracion-totales
- Ingresos


In [4]:
totales_VMD = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Totales VMD')

penetracion_totales = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Penetracion-totales')
# Por alguna razon en este dataset Año y trimestre los reconoce como string y debo castearlos a numericos
penetracion_totales = penetracion_totales.with_columns(pl.col('Año').cast(pl.Int64).alias('Año'))
penetracion_totales = penetracion_totales.with_columns(pl.col('Trimestre').cast(pl.Int64).alias('Trimestre'))

ingresos = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Ingresos ')

In [5]:
totales_VMD.head(1)

Año,Trimestre,Mbps (Media de bajada),Trimestre_1
i64,i64,f64,str
2024,1,139.15,"""Ene-Mar 2024"""


In [6]:
penetracion_totales.head(1)

Año,Trimestre,Accesos por cada 100 hogares,Accesos por cada 100 hab,Periodo
i64,i64,f64,f64,str
2024,1,78.89,24.79,"""Ene-Mar 2024"""


In [7]:
ingresos.head(1)

Año,Trimestre,Ingresos (miles de pesos),Periodo
i64,i64,f64,str
2024,1,2.8042e8,"""Ene-Mar 2024"""


Debo Crear una Columna a los 3 dataframes que me sirva para poder unir la información luego con un simple `Join`

In [8]:
totales_VMD = totales_VMD.with_columns((pl.col('Año') * 100 + pl.col('Trimestre')).alias('id'))

penetracion_totales = penetracion_totales.with_columns((pl.col('Año') * 100 + pl.col('Trimestre')).alias('id'))

ingresos = ingresos.with_columns((pl.col('Año') * 100 + pl.col('Trimestre')).alias('id'))

In [10]:
totales_VMD.head(1)

Año,Trimestre,Mbps (Media de bajada),Trimestre_1,id
i64,i64,f64,str,i64
2024,1,139.15,"""Ene-Mar 2024""",202401


In [11]:
penetracion_totales.head(1)

Año,Trimestre,Accesos por cada 100 hogares,Accesos por cada 100 hab,Periodo,id
i64,i64,f64,f64,str,i64
2024,1,78.89,24.79,"""Ene-Mar 2024""",202401


In [12]:
ingresos.head(1)

Año,Trimestre,Ingresos (miles de pesos),Periodo,id
i64,i64,f64,str,i64
2024,1,2.8042e8,"""Ene-Mar 2024""",202401


Ahora si puedo crear la tabla `internet_pais` a partir de cualquiera de estos dataframe y utilizar la columna `id` para aplicar `join` y recopilar la inf en un solo dataframe

In [13]:
internet_pais = ingresos[['id','Año','Trimestre','Periodo','Ingresos (miles de pesos)']]

internet_pais = internet_pais.join(
    penetracion_totales[['id','Accesos por cada 100 hogares', 'Accesos por cada 100 hab']],\
    on='id', how='left', coalesce=True
)

internet_pais.head(1)

id,Año,Trimestre,Periodo,Ingresos (miles de pesos),Accesos por cada 100 hogares,Accesos por cada 100 hab
i64,i64,i64,str,f64,f64,f64
202401,2024,1,"""Ene-Mar 2024""",2.8042e8,78.89,24.79


In [14]:
internet_pais = internet_pais.join(totales_VMD[['id','Mbps (Media de bajada)']],\
                                   on='id', how='left', coalesce=True
)
internet_pais.head(1)

id,Año,Trimestre,Periodo,Ingresos (miles de pesos),Accesos por cada 100 hogares,Accesos por cada 100 hab,Mbps (Media de bajada)
i64,i64,i64,str,f64,f64,f64,f64
202401,2024,1,"""Ene-Mar 2024""",2.8042e8,78.89,24.79,139.15


Si renombramos las  columnas y eliminamos Periodo, ya tenemos la tabla `internet_pais`

In [15]:
internet_pais = internet_pais.drop('Periodo')
internet_pais.columns = ['id','anio','trimestre','ingresos_miles','accesos_100_hog', 'accesos_100_hab', 'mbps_media_bajada']

print(f'Contidad de Registros {internet_pais.shape[0]}')
internet_pais.head(5)

Contidad de Registros 41


id,anio,trimestre,ingresos_miles,accesos_100_hog,accesos_100_hab,mbps_media_bajada
i64,i64,i64,f64,f64,f64,f64
202401,2024,1,2.8042e8,78.89,24.79,139.15
202304,2023,4,1.6738e8,78.56,24.66,139.04
202303,2023,3,1.3311e8,77.84,24.41,129.67
202302,2023,2,1.1806e8,77.02,24.14,123.95
202301,2023,1,1.0413e8,77.2,24.17,110.5


In [16]:
del penetracion_totales
del totales_VMD
del ingresos

***
## **Creando la Tabla `internet_provincia`**
Para la creacion de esta tabla necesito los siguientes datasets
- Penetracion-hogares
- Penetración-poblacion
- Velocidad % por prov
- Dial-BAf

In [17]:
penetracion_hogares = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Penetracion-hogares')

penetracion_poblacion = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Penetración-poblacion')

velocidad_porcent_por_prov = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Velocidad % por prov')

dial_BAf = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Dial-BAf')

In [18]:
penetracion_hogares.head(1)

Año,Trimestre,Provincia,Accesos por cada 100 hogares
i64,i64,str,f64
2024,1,"""Buenos Aires""",81.1


In [19]:
penetracion_poblacion.head(1)

Año,Trimestre,Provincia,Accesos por cada 100 hab
i64,i64,str,f64
2024,1,"""Buenos Aires""",27.8


In [20]:
velocidad_porcent_por_prov.head(1)

Año,Trimestre,Provincia,Mbps (Media de bajada)
i64,i64,str,f64
2024,1,"""Buenos Aires""",159.13


In [21]:
dial_BAf.head(1)

Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
i64,i64,str,i64,i64,i64
2024,1,"""Buenos Aires""",5038657,5900,5044557


Debo estar seguro de que los nombres de Provincia en los 4 dataframe anteriores coinciden exactamente para poder juntar la información, para ello voy a normalizar los nombres de esa columna con las variables `reemplazar` y `correccion` y luego introduciré el `id_provincia` de la tabla `provincia`, una vez que logré hacer esto a cabalidad estaré seguro de dar el siguiente paso. 

In [22]:
# Normalizando los nombres de las Provincias en los 4 dataframe
for i in range(len(reemplazar)):
    penetracion_hogares = penetracion_hogares.with_columns(
        pl.when(pl.col('Provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('Provincia'))
          .alias('Provincia')
    )

    penetracion_poblacion = penetracion_poblacion.with_columns(
        pl.when(pl.col('Provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('Provincia'))
          .alias('Provincia')
    )

    dial_BAf = dial_BAf.with_columns(
        pl.when(pl.col('Provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('Provincia'))
          .alias('Provincia')
    )

    velocidad_porcent_por_prov = velocidad_porcent_por_prov.with_columns(
        pl.when(pl.col('Provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('Provincia'))
          .alias('Provincia')
    )

Necesito introducir en cada uno de estos dataframe el `id_provincia`, pero para esto es imprescindible que coincidan los nombres de las columnas provincia de todas las tablas

In [23]:
# Cambiando de Provincia a provincia en todos los dataframes
penetracion_hogares = penetracion_hogares.rename({'Provincia': 'provincia'})

penetracion_poblacion = penetracion_poblacion.rename({'Provincia': 'provincia'})

dial_BAf = dial_BAf.rename({'Provincia': 'provincia'})

velocidad_porcent_por_prov = velocidad_porcent_por_prov.rename({'Provincia': 'provincia'})

incorporo a todos el `id_provincia`

In [24]:
penetracion_hogares = penetracion_hogares.join(
    provincias[['id_provincia','provincia']], on='provincia', how='left', coalesce=True 
)

penetracion_poblacion = penetracion_poblacion.join(
    provincias[['id_provincia','provincia']], on='provincia', how='left', coalesce=True 
)

dial_BAf = dial_BAf.join(
    provincias[['id_provincia','provincia']], on='provincia', how='left', coalesce=True 
)

velocidad_porcent_por_prov = velocidad_porcent_por_prov.join(
    provincias[['id_provincia','provincia']], on='provincia', how='left', coalesce=True 
)

Ahora creo la columna `id` por donde voy a hacer los `join`


In [25]:
'''
Ejemplo:
Año 2024, Trimestre 1, id_provincia 2
(2024 * 100000) + (1 * 1000) + (2)  ==> id = 202401002
'''

penetracion_hogares = penetracion_hogares.with_columns(
    ((pl.col('Año') * 100000) + (pl.col('Trimestre') * 1000) + (pl.col('id_provincia'))).alias('id')
)

penetracion_poblacion = penetracion_poblacion.with_columns(
    ((pl.col('Año') * 100000) + (pl.col('Trimestre') * 1000) + (pl.col('id_provincia'))).alias('id')
)

dial_BAf = dial_BAf.with_columns(
    ((pl.col('Año') * 100000) + (pl.col('Trimestre') * 1000) + (pl.col('id_provincia'))).alias('id')
)

velocidad_porcent_por_prov = velocidad_porcent_por_prov.with_columns(
    ((pl.col('Año') * 100000) + (pl.col('Trimestre') * 1000) + (pl.col('id_provincia'))).alias('id')
)

In [26]:
penetracion_poblacion.head(1)

Año,Trimestre,provincia,Accesos por cada 100 hab,id_provincia,id
i64,i64,str,f64,i64,i64
2024,1,"""Buenos Aires""",27.8,6,202401006


In [27]:
penetracion_hogares.head(1)

Año,Trimestre,provincia,Accesos por cada 100 hogares,id_provincia,id
i64,i64,str,f64,i64,i64
2024,1,"""Buenos Aires""",81.1,6,202401006


In [28]:
dial_BAf.head(1)

Año,Trimestre,provincia,Banda ancha fija,Dial up,Total,id_provincia,id
i64,i64,str,i64,i64,i64,i64,i64
2024,1,"""Buenos Aires""",5038657,5900,5044557,6,202401006


In [29]:
velocidad_porcent_por_prov.head(1)

Año,Trimestre,provincia,Mbps (Media de bajada),id_provincia,id
i64,i64,str,f64,i64,i64
2024,1,"""Buenos Aires""",159.13,6,202401006


Ahora si puedo crear la tabla `internet_provincia` a partir de cualquiera de estos dataframe y utilizar la columna `id` para aplicar `join` y recopilar la inf en un solo dataframe

In [30]:
internet_provincia = velocidad_porcent_por_prov[['id', 'Año', 'Trimestre', 'id_provincia', 'Mbps (Media de bajada)']]

internet_provincia = internet_provincia.join(
    penetracion_hogares[['id', 'Accesos por cada 100 hogares']], on='id', how='left', coalesce=True
)

internet_provincia = internet_provincia.join(
    penetracion_poblacion[['id', 'Accesos por cada 100 hab']], on='id', how='left', coalesce=True
)

internet_provincia = internet_provincia.join(
    dial_BAf[['id', 'Banda ancha fija', 'Dial up']], on='id', how='left', coalesce=True
)

internet_provincia.head()

id,Año,Trimestre,id_provincia,Mbps (Media de bajada),Accesos por cada 100 hogares,Accesos por cada 100 hab,Banda ancha fija,Dial up
i64,i64,i64,i64,f64,f64,f64,i64,i64
202401006,2024,1,6,159.13,81.1,27.8,5038657,5900
202401002,2024,1,2,229.94,119.53,48.78,1502661,2050
202401010,2024,1,10,91.45,68.82,17.5,74932,1
202401022,2024,1,22,107.51,46.7,12.48,155538,5
202401026,2024,1,26,20.5,84.89,26.02,169819,857


vamos a renombrar las columnas de la tabla `internet_provincia`

In [31]:
columns = ['id', 'anio', 'trimestre', 'id_provincia', 'mbps_media_bajada', 'accesos_100_hog',
           'accesos_100_hab', 'accesos_banda_ancha', 'accesos_dial_up']

internet_provincia.columns = columns

internet_provincia.head(5)

id,anio,trimestre,id_provincia,mbps_media_bajada,accesos_100_hog,accesos_100_hab,accesos_banda_ancha,accesos_dial_up
i64,i64,i64,i64,f64,f64,f64,i64,i64
202401006,2024,1,6,159.13,81.1,27.8,5038657,5900
202401002,2024,1,2,229.94,119.53,48.78,1502661,2050
202401010,2024,1,10,91.45,68.82,17.5,74932,1
202401022,2024,1,22,107.51,46.7,12.48,155538,5
202401026,2024,1,26,20.5,84.89,26.02,169819,857


Hay algunos nulos que van a provocar errores al vaciar la data en una tabla sql

In [32]:
for column in internet_provincia.columns:
    print(f'Column ----> {column}')
    print(f'Nulos ----> {internet_provincia[column].is_null().sum()}')
    print('*'*80)

Column ----> id
Nulos ----> 0
********************************************************************************
Column ----> anio
Nulos ----> 0
********************************************************************************
Column ----> trimestre
Nulos ----> 0
********************************************************************************
Column ----> id_provincia
Nulos ----> 0
********************************************************************************
Column ----> mbps_media_bajada
Nulos ----> 0
********************************************************************************
Column ----> accesos_100_hog
Nulos ----> 0
********************************************************************************
Column ----> accesos_100_hab
Nulos ----> 0
********************************************************************************
Column ----> accesos_banda_ancha
Nulos ----> 0
********************************************************************************
Column ----> accesos_dial_up
Nulos 

Veamos si es Normal dentro de esta columna tener valores con 0

In [33]:
mask = internet_provincia['accesos_dial_up'] == 0
print(f'Cantidad de Registros con 0 en columna "accesos_dial_up" --> {internet_provincia.filter(mask).shape[0]}')

Cantidad de Registros con 0 en columna "accesos_dial_up" --> 80


Pasamos a 0 estos 2 registros que vinieron en Blanco y los tomó como Nulos y ya estaría lista la tabla

In [34]:
internet_provincia = internet_provincia.with_columns(pl.col('accesos_dial_up').fill_null(0))
internet_provincia.head()

id,anio,trimestre,id_provincia,mbps_media_bajada,accesos_100_hog,accesos_100_hab,accesos_banda_ancha,accesos_dial_up
i64,i64,i64,i64,f64,f64,f64,i64,i64
202401006,2024,1,6,159.13,81.1,27.8,5038657,5900
202401002,2024,1,2,229.94,119.53,48.78,1502661,2050
202401010,2024,1,10,91.45,68.82,17.5,74932,1
202401022,2024,1,22,107.51,46.7,12.48,155538,5
202401026,2024,1,26,20.5,84.89,26.02,169819,857


In [35]:
del dial_BAf
del velocidad_porcent_por_prov
del penetracion_hogares
del penetracion_poblacion

***
## **Creando la Tabla `accesos_x_tecnologia`**


In [36]:
accesos_x_tecnologia = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Accesos Por Tecnología')
accesos_x_tecnologia.head()

Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
i64,i64,str,i64,i64,i64,i64,i64,i64
2024,1,"""Buenos Aires""",203297,2790663,1799013,150290,101294,5044557
2024,1,"""Capital Federal""",70108,1177475,203756,4493,48879,1504711
2024,1,"""Catamarca""",5901,11025,56542,1384,81,74933
2024,1,"""Chaco""",11343,58929,64463,8404,12404,155543
2024,1,"""Chubut""",26306,77217,26356,30867,9930,170676


Cambiando los nombres de las columnas y normalizando los nombres de las Provincias

In [37]:
columns = ['anio', 'trimestre', 'provincia', 'adsl', 'cablemoden', 'fibra', 'wireless', 'otros', 'total']

accesos_x_tecnologia.columns = columns

for i in range(len(reemplazar)):
    accesos_x_tecnologia = accesos_x_tecnologia.with_columns(
        pl.when(pl.col('provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('provincia'))
          .alias('provincia')
    )

accesos_x_tecnologia.head()

anio,trimestre,provincia,adsl,cablemoden,fibra,wireless,otros,total
i64,i64,str,i64,i64,i64,i64,i64,i64
2024,1,"""Buenos Aires""",203297,2790663,1799013,150290,101294,5044557
2024,1,"""Ciudad Autónoma de Buenos Aire…",70108,1177475,203756,4493,48879,1504711
2024,1,"""Catamarca""",5901,11025,56542,1384,81,74933
2024,1,"""Chaco""",11343,58929,64463,8404,12404,155543
2024,1,"""Chubut""",26306,77217,26356,30867,9930,170676


Falta colocar el respectivo `id_provincia` y borrar la columna provincia

In [38]:
accesos_x_tecnologia = accesos_x_tecnologia.join(
    provincias[['id_provincia','provincia']], on='provincia', how='left', coalesce=True
)

accesos_x_tecnologia = accesos_x_tecnologia.drop('provincia')

# Creando el id de la Tabla.  Misma Metodología de la tabla anterior.
accesos_x_tecnologia = accesos_x_tecnologia.with_columns(
    ((pl.col('anio') * 100000) + (pl.col('trimestre') * 1000) + (pl.col('id_provincia'))).alias('id')
)

accesos_x_tecnologia.head()

anio,trimestre,adsl,cablemoden,fibra,wireless,otros,total,id_provincia,id
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2024,1,203297,2790663,1799013,150290,101294,5044557,6,202401006
2024,1,70108,1177475,203756,4493,48879,1504711,2,202401002
2024,1,5901,11025,56542,1384,81,74933,10,202401010
2024,1,11343,58929,64463,8404,12404,155543,22,202401022
2024,1,26306,77217,26356,30867,9930,170676,26,202401026


***
## **Creando la Tabla `velocidad_sin_rangos`**

In [39]:
df = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Velocidad_sin_Rangos')
df.head(3)

Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,2 Mbps","1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps","6,7 Mbps",3 Mbps,"3,2 Mbps","3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,1 Mbps","10,2 Mbps","10,6 Mbps",11 Mbps,12 Mbps,…,350 Mbps,380 Mbps,384 Mbps,400 Mbps,450 Mbps,473 Mbps,480 Mbps,500 Mbps,512 Mbps,520 Mbps,540 Mbps,600 Mbps,640 Mbps,680 Mbps,700 Mbps,768 Mbps,800 Mbps,850 Mbps,900 Mbps,945 Mbps,950 Mbps,999 Mbps,1000 Mbps,1024 Mbps,1075 Mbps,1536 Mbps,2000 Mbps,2048 Mbps,2529 Mbps,3072 Mbps,4000 Mbps,5000 Mbps,5125 Mbps,6000 Mbps,7000 Mbps,8000 Mbps,10000 Mbps
i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2024,1,"""Buenos Aires""",125147,18,0,0,25815,169,4611,17899,0,0,141,19520,0,21,0,43554,0,0,41735,13365,83,27327,75439,0,11286,53,63826,688,157372,0,0,0,10,21114,…,0,0,0,965,0,0,5,149680,663,0,0,12163,0,0,0,0,563,0,0,0,0,26,156691,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2024,1,"""Capital Federal""",0,31,0,0,451,34,1307,4111,0,0,0,249,0,0,0,3971,0,0,11975,994,0,3547,9017,0,7,0,16864,3,34541,0,0,0,5,92,…,0,0,0,3,0,0,0,86440,0,0,0,372,0,0,1,0,1,0,0,0,0,25,68405,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2024,1,"""Catamarca""",70,0,0,0,34,37,0,176,0,0,0,115,0,0,0,1644,0,0,0,1,0,332,780,0,0,0,0,0,3424,0,0,0,0,31,…,0,0,0,0,0,0,0,372,0,0,0,834,0,0,0,0,0,0,0,0,0,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
mask = df['Año'].is_null()
df.filter(mask)

Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,2 Mbps","1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps","6,7 Mbps",3 Mbps,"3,2 Mbps","3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,1 Mbps","10,2 Mbps","10,6 Mbps",11 Mbps,12 Mbps,…,350 Mbps,380 Mbps,384 Mbps,400 Mbps,450 Mbps,473 Mbps,480 Mbps,500 Mbps,512 Mbps,520 Mbps,540 Mbps,600 Mbps,640 Mbps,680 Mbps,700 Mbps,768 Mbps,800 Mbps,850 Mbps,900 Mbps,945 Mbps,950 Mbps,999 Mbps,1000 Mbps,1024 Mbps,1075 Mbps,1536 Mbps,2000 Mbps,2048 Mbps,2529 Mbps,3072 Mbps,4000 Mbps,5000 Mbps,5125 Mbps,6000 Mbps,7000 Mbps,8000 Mbps,10000 Mbps
i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


El df al cargar se trajo una linea del final del dataset con informacion nula.. Hay que borrarla

In [41]:
df = df.filter(~mask)

Normalizando la escritura de las Provincias

In [42]:
for i in range(len(reemplazar)):
    df = df.with_columns(
        pl.when(pl.col('Provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('Provincia'))
          .alias('Provincia')
    )

Creando a `velocidad_sin_rangos` con solo 5 columnas

In [43]:
"""
columna 0 ---> Año
columna 1 ---> Trimestre
columna 2 ---> Provincia
"""

# tomo las columnas de "Otros" en adelante
columns = df.columns[3:]
list_velocidad_sin_rangos = []
for i in range(df.shape[0]):
    for j,column in enumerate(columns):
        if df[i,3+j] != 0:
            row = {}
            row['anio'] = df[i,0]
            row['trimestre'] = df[i,1]
            row['provincia'] = df[i,2]
            row['velocidad_mbps'] = -10 if column == 'Otros' else float(column[:column.find(' ')].replace(',','.'))
            row['accesos'] = df[i,3+j]
            list_velocidad_sin_rangos.append(row)
 
velocidad_sin_rangos = pl.DataFrame(list_velocidad_sin_rangos)
# Verificando cuantas velocidades distintas existen
print(f"{velocidad_sin_rangos['velocidad_mbps'].unique().count()} distintas velocidades")
velocidad_sin_rangos.head(6)

188 distintas velocidades


anio,trimestre,provincia,velocidad_mbps,accesos
i64,i64,str,f64,i64
2024,1,"""Buenos Aires""",-10.0,125147
2024,1,"""Buenos Aires""",0.256,18
2024,1,"""Buenos Aires""",0.5,25815
2024,1,"""Buenos Aires""",0.512,169
2024,1,"""Buenos Aires""",0.75,4611
2024,1,"""Buenos Aires""",1.0,17899


<img src='assets/imagen_01.png'><br>

Cuando en el Dataframe la `velocidad` es `-10` se refiere a `otros` y notar que solo toma las celdas cuyos accesos no son cero (0).  El dataframe inicial tiene 193 columnas 3 para (Año, Trimestre, provincia) y 190 columnas para las velocidades de conexion. En las columnas `velocidad` y `accesos` está la información de las `190 distintas columnas de velocidades` que hay.  Arriba indica que hay sólo `188 velocidades distintas`, porque en las columnas de `6,7 MBPS` y `86 MBPS` no hay ningún dato, lo que verifica que el algoritmo funcionó bien.

Ahora debo hacer un Join con la tabla `provincias` para incorporar el id_provincia, luego borro a `provincia`

In [44]:
velocidad_sin_rangos = velocidad_sin_rangos.join(
    provincias[['id_provincia', 'provincia']], on='provincia', how='left', coalesce=True
)

velocidad_sin_rangos = velocidad_sin_rangos.drop('provincia')

velocidad_sin_rangos = velocidad_sin_rangos.sort('anio', 'trimestre')

velocidad_sin_rangos.head(10)

anio,trimestre,velocidad_mbps,accesos,id_provincia
i64,i64,f64,i64,i64
2017,4,150.0,4367,90
2017,4,30.0,1303,90
2017,4,20.0,24242,90
2017,4,15.0,76,90
2017,4,10.0,45241,90
2017,4,6.0,70565,90
2017,4,5.0,9,90
2017,4,3.5,8017,90
2017,4,3.0,159,90


Para esta última tabla el `id` debo dejar que lo cree el motor sql de manera autoincremental. Aca la formula de las pasadas 2 tablas NO funciona dado que para una misma combinación de ('anio' + 'trimestre' + 'id_provincia') existen distintas velocidades. Le dí un orden cronológico para que el motor de Sql tome primero los registros con fechas mas antigua 

***
## **Creando la Tabla `accesos_localidades`**

Cargando los datasets de donde voy a obtener los datos

In [45]:
mapa_conect = pl.read_excel('datasets_iniciales/mapa_conectividad.xlsx', sheet_name='Hoja3')

accesos_localidades = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Accesos_tecnologia_localidad')

**Haciendo ajustes en `accesos_localidades`**

In [46]:
accesos_localidades.tail()

Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general
str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64
null,null,null,null,0,0,0,0,0,0,0,0,0
null,null,null,null,0,0,0,0,0,0,0,0,0
null,null,null,null,0,0,0,0,0,0,0,0,0
null,null,null,null,0,0,0,0,0,0,0,0,0
null,null,null,null,0,0,0,0,0,0,0,0,0


Fueron Lineas al final de los datos en la Hoja de excel que cargo el Dataframe sin informacion... en las columnas que no aparecen Nulos esas lineas es porque en la hoja de excel tenian `-` y los tomo como 0. Solucion Hay que borrar esas lineas

In [47]:
accesos_localidades = accesos_localidades.with_columns(pl.col('Provincia').str.to_titlecase().alias('Provincia'))

for i in range(len(reemplazar)):
    accesos_localidades = accesos_localidades.with_columns(
        pl.when(pl.col('Provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('Provincia'))
          .alias('Provincia')
    )

In [48]:
# El Dataframe Cargo unos NULOS sin informacion. Lineas con rayas `-` al final de la hoja de calculo
mask = accesos_localidades['Provincia'].is_null()
accesos_localidades_2 = accesos_localidades.filter(~mask)

# filtrando en la Hoja de Calculo Aquellas filas sin Link lo que aca llamamos id_indec tienen son sub localidades
# con demasiado pocas conexiones.  muchas veces 1 conexion. 
mask = accesos_localidades['Link Indec'] == 'Sin Datos'
accesos_localidades = accesos_localidades.filter(~mask)

# renombro columnas
accesos_localidades.columns = ['Provincia', 'Partido', 'Localidad','id_indec', 'adsl', 'cablemodem',
                               'dial_up', 'fibra', 'otros', 'satelital', 'winmax', 'wireless', 'total']

# Casteo a numerico el id_indec
accesos_localidades = accesos_localidades.with_columns(pl.col('id_indec').cast(pl.Int64).alias('id_indec'))

accesos_localidades.head(1)

Provincia,Partido,Localidad,id_indec,adsl,cablemodem,dial_up,fibra,otros,satelital,winmax,wireless,total
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,755,4600,0,2,0,742,0,727,6826


**Ahora hagamos ajustes en `mapa_conect`**

In [50]:
mapa_conect.head(1)

Provincia,Partido,Localidad,Población,ADSL,Cablemódem,Dial Up,Fibra óptica,Satelital,Wireless,Telefonía Fija,3G,4G,Link,Latitud,Longitud
str,str,str,i64,str,str,str,str,str,str,str,str,str,i64,str,str
"""BUENOS AIRES""","""25 de Mayo""","""25 de Mayo""",23408,"""SI""","""SI""","""--""","""--""","""SI""","""SI""","""SI""","""SI""","""SI""",6854100,"""-35.4339385892588""","""-60.1731209454056"""


In [51]:
mapa_conect = mapa_conect.with_columns(pl.col('Provincia').str.to_titlecase().alias('Provincia'))

for i in range(len(reemplazar)):
    mapa_conect = mapa_conect.with_columns(
        pl.when(pl.col('Provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('Provincia'))
          .alias('Provincia')
    )

In [52]:
columns = [ ['3G', '4G'], ['Latitud', 'Longitud'] ]

# en 3G y 4G vamos a sustituir "SI" por True y lo demas por False en una nueva columna
# y vamos a castear las columnas de las coordenadas 
for i in range(2):
    mapa_conect = mapa_conect.with_columns([
        pl.when( pl.col(columns[0][i]) == 'SI')
          .then(pl.lit(True))
          .otherwise(pl.lit(False))
          .alias(columns[0][i].lower()),
        
        pl.col(columns[1][i]).cast(pl.Float64).alias(columns[1][i].lower())  
    ])


# Borremos las antiguas columnas de coordenadas
mapa_conect = mapa_conect.drop(['Latitud', 'Longitud', '3G', '4G'])

mapa_conect.head(1)

Provincia,Partido,Localidad,Población,ADSL,Cablemódem,Dial Up,Fibra óptica,Satelital,Wireless,Telefonía Fija,Link,3g,latitud,4g,longitud
str,str,str,i64,str,str,str,str,str,str,str,i64,bool,f64,bool,f64
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",23408,"""SI""","""SI""","""--""","""--""","""SI""","""SI""","""SI""",6854100,true,-35.433939,true,-60.173121


In [53]:
# Renombro la columna que uso para el join
mapa_conect = mapa_conect.rename({'Link': 'id_indec', 'Población': 'poblacion'})

# Borro algunas columnas
mapa_conect = mapa_conect.drop(['ADSL','Cablemódem','Dial Up', 'Fibra óptica','Satelital','Wireless','Telefonía Fija'])

mapa_conect.head(1)

Provincia,Partido,Localidad,poblacion,id_indec,3g,latitud,4g,longitud
str,str,str,i64,i64,bool,f64,bool,f64
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",23408,6854100,true,-35.433939,true,-60.173121


In [54]:
accesos_localidades.shape

(3002, 13)

**Ahora hay que hacer un `join` entre los 2 Datasets**

In [55]:
accesos_localidades = accesos_localidades.join(
    mapa_conect, on=['Provincia', 'Partido', 'Localidad'], how='left', coalesce=True
)

print(accesos_localidades.shape)
accesos_localidades.head()

(3002, 19)


Provincia,Partido,Localidad,id_indec,adsl,cablemodem,dial_up,fibra,otros,satelital,winmax,wireless,total,poblacion,id_indec_right,3g,latitud,4g,longitud
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,f64,bool,f64
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,755,4600,0,2,0,742,0,727,6826,23408,6854100,true,-35.433939,true,-60.173121
"""Buenos Aires""","""25 de Mayo""","""Del Valle""",6854020,181,0,0,0,10,1,0,0,192,899,6854020,true,-35.897345,false,-60.731609
"""Buenos Aires""","""25 de Mayo""","""Gobernador Ugarte""",6854040,0,0,0,0,0,0,0,181,181,547,6854040,false,-35.164397,false,-60.081358
"""Buenos Aires""","""25 de Mayo""","""Norberto de la Riestra""",6854060,0,782,0,496,16,0,0,185,1479,4524,6854060,true,-35.272779,true,-59.771933
"""Buenos Aires""","""25 de Mayo""","""Lucas Monteverde""",6854050,0,0,0,0,0,0,0,6,6,63,6854050,false,-35.483827,false,-59.988065


Borro la Columna `id_indec_right` y renombro otras

In [56]:
accesos_localidades = accesos_localidades.drop('id_indec_right')

accesos_localidades = accesos_localidades.rename(
    {'Provincia':'provincia', 'Partido': 'partido', 'Localidad': 'localidad','3g': '_3g', '4g': '_4g'}
)


Observo sí hubo coincidencias NULAS

In [58]:
for column in accesos_localidades.columns:
    print(f'Columna --> {column}')
    print(f'Cantidad de Nulos --> {accesos_localidades[column].is_null().sum()}')
    print('*'*50)

Columna --> provincia
Cantidad de Nulos --> 0
**************************************************
Columna --> partido
Cantidad de Nulos --> 0
**************************************************
Columna --> localidad
Cantidad de Nulos --> 0
**************************************************
Columna --> id_indec
Cantidad de Nulos --> 0
**************************************************
Columna --> adsl
Cantidad de Nulos --> 0
**************************************************
Columna --> cablemodem
Cantidad de Nulos --> 0
**************************************************
Columna --> dial_up
Cantidad de Nulos --> 0
**************************************************
Columna --> fibra
Cantidad de Nulos --> 0
**************************************************
Columna --> otros
Cantidad de Nulos --> 0
**************************************************
Columna --> satelital
Cantidad de Nulos --> 0
**************************************************
Columna --> winmax
Cantidad de Nulos --> 0
****

In [59]:
mask = accesos_localidades['poblacion'].is_null()
accesos_localidades.filter(mask)

provincia,partido,localidad,id_indec,adsl,cablemodem,dial_up,fibra,otros,satelital,winmax,wireless,total,poblacion,_3g,latitud,_4g,longitud
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,f64,bool,f64
"""Buenos Aires""","""9 de Julio""","""Alfredo Demarchi (Est. Facund…",6588010,450,0,0,0,60,0,0,0,510,null,null,null,null,null
"""Buenos Aires""","""9 de Julio""","""Manuel B. Gonnet (Est. French…",6588060,193,0,0,2,2,0,0,0,197,null,null,null,null,null
"""Buenos Aires""","""9 de Julio""","""Villa General Fournier (Est. …",6588120,0,0,0,118,0,0,0,42,160,null,null,null,null,null
"""Buenos Aires""","""Adolfo Gonzales Chaves""","""Adolfo Gonzales Chaves (Est. …",6014010,464,0,2,943,0,0,0,690,2099,null,null,null,null,null
"""Buenos Aires""","""Alberti""","""Alberti (Est. Andrés Vaccare…",6021010,335,0,6,1,0,0,0,2,344,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Neuquén""","""Minas""","""Villa Del Nahueve""",58077070,0,0,0,0,0,0,0,15,15,null,null,null,null,null
"""Salta""","""Rosario de Lerma""","""La Merced Del Encón""",66147015,0,0,0,175,0,0,0,82,257,null,null,null,null,null
"""San Juan""","""Santa Lucía""","""Alto de Sierra""",70098010,0,0,0,0,0,0,0,88,88,null,null,null,null,null


Estos `87 Nulos` son localidades que aparecen en la Data de la Hoja de `Accesos_tecnologia_localidad` del archivo de internet, pero que `NO APARECEN` en el archivo de `Mapa de Conectividad`. Aspectos a considerar.<br>
- Sí el Ente Nacional de Comunicaciones (ENACOM) de Argentina, no las tomó en cuenta en su `Mapa de Conectividad` desconozco con certeza las razones, pero presumo que se puede deber a Localidades Rurales de muy POCA POBLACION, dada la cantidad de Conecciones que se observan.
- Sí Nuestro cliente es un Potencial Inversionista en le Segmento de Ofertar Servicios de Internet, pienso que por la Inversión Inicial que debe hacer, seguramente va a preferir enfocarse en localidades mas Pobladas (mas potenciales clientes), con mejor infraestructura, ya que estas que excluye el ENACOM en algunos casos no llegan a 2000 Habitantes.  
- No pretendo decir que sean poco importantes, pero sí el ENACOM no las consideró en su mapa de conectividad, no deberían ser consideradas como las primeras opciones de localidades donde invertir.
- Por ahora las voy a dejar porque necesito crear un Maestro de `partidos` y `localidades` despues que cree esas dos tablas y guarde la información de esas localidades, borraré de ESTA TABLA esas localidades  

***
## **Creando la Tabla `rangos_localidades`**

In [64]:
rangos_localidades_2 = pl.read_excel('datasets_iniciales/Internet.xlsx', sheet_name='Acc_vel_loc_sinrangos')

rangos_localidades_2.head()

Provincia,Partido,Localidad,Link Indec,Otros,"0,256 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps",3 Mbps,"3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,1 Mbps","10,2 Mbps",11 Mbps,12 Mbps,"12,3 Mbps",13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,…,160 Mbps,164 Mbps,165 Mbps,180 Mbps,200 Mbps,205 Mbps,212 Mbps,220 Mbps,225 Mbps,240 Mbps,246 Mbps,250 Mbps,256 Mbps,275 Mbps,300 Mbps,320 Mbps,325 Mbps,330 Mbps,350 Mbps,400 Mbps,450 Mbps,480 Mbps,500 Mbps,512 Mbps,520 Mbps,540 Mbps,600 Mbps,680 Mbps,700 Mbps,800 Mbps,850 Mbps,900 Mbps,999 Mbps,1000 Mbps,1024 Mbps,4000 Mbps,10000 Mbps
str,str,str,str,i64,i64,i64,i64,i64,i64,null,i64,null,null,i64,null,i64,i64,null,i64,i64,null,i64,null,i64,i64,i64,null,null,i64,i64,null,i64,i64,i64,i64,i64,…,null,null,null,null,i64,null,null,null,null,null,null,i64,null,null,i64,null,null,null,null,null,null,null,i64,null,null,null,i64,null,null,null,null,null,i64,i64,null,null,null
"""BUENOS AIRES""","""25 de Mayo""","""25 de Mayo""","""6854100""",null,null,2,null,19,null,null,null,null,null,85,null,145,null,null,null,76,null,null,null,299,null,288,null,null,null,null,null,null,null,415,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,95,null,null,null,null,null,null,null,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""BUENOS AIRES""","""25 de Mayo""","""Agustín Mosconi""","""6854010""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,59,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""BUENOS AIRES""","""25 de Mayo""","""Del Valle""","""6854020""",null,null,1,null,null,null,null,null,null,null,181,null,null,null,null,null,null,null,null,null,null,null,10,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""BUENOS AIRES""","""25 de Mayo""","""Ernestina""","""6854030""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,75,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""BUENOS AIRES""","""25 de Mayo""","""Gobernador Ugarte""","""6854040""",null,null,null,null,null,null,null,106,null,null,3,null,null,56,null,7,3,null,null,null,4,null,1,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


- Normalizemos Provincia
- Convirtamos los Nulos en 0 en las columnas que van desde `Otros` hasta `10000 Mbps`
- Cambiemos Nombre a ciertas columnas
- Al igual que el Dataset anterior y por las mismas razones eliminemos los que vienen con `Sin Datos`
- Castear a Númerico el `id_indec`

In [65]:
rangos_localidades_2 = rangos_localidades_2.with_columns(pl.col('Provincia').str.to_titlecase().alias('Provincia'))

for i in range(len(reemplazar)):
    rangos_localidades_2 = rangos_localidades_2.with_columns(
        pl.when(pl.col('Provincia').is_in(reemplazar[i]))
          .then(pl.lit(correcion[i]))
          .otherwise(pl.col('Provincia'))
          .alias('Provincia')
    )

rangos_localidades_2 = rangos_localidades_2.rename(
    {'Provincia':'provincia', 'Partido': 'partido', 'Localidad': 'localidad', 'Link Indec': 'id_indec'}
)    

columns = rangos_localidades_2.columns[4:]
for column in columns:
    rangos_localidades_2 = rangos_localidades_2.with_columns(pl.col(column).fill_null(0).alias(column))

mask = rangos_localidades_2['id_indec'] == 'Sin Datos'
rangos_localidades_2 = rangos_localidades_2.filter(~mask)

rangos_localidades_2 = rangos_localidades_2.with_columns(
    pl.col('id_indec').cast(pl.Int64).alias('id_indec')
)

rangos_localidades_2.head()

provincia,partido,localidad,id_indec,Otros,"0,256 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps",3 Mbps,"3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,1 Mbps","10,2 Mbps",11 Mbps,12 Mbps,"12,3 Mbps",13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,…,160 Mbps,164 Mbps,165 Mbps,180 Mbps,200 Mbps,205 Mbps,212 Mbps,220 Mbps,225 Mbps,240 Mbps,246 Mbps,250 Mbps,256 Mbps,275 Mbps,300 Mbps,320 Mbps,325 Mbps,330 Mbps,350 Mbps,400 Mbps,450 Mbps,480 Mbps,500 Mbps,512 Mbps,520 Mbps,540 Mbps,600 Mbps,680 Mbps,700 Mbps,800 Mbps,850 Mbps,900 Mbps,999 Mbps,1000 Mbps,1024 Mbps,4000 Mbps,10000 Mbps
str,str,str,i64,i64,i64,i64,i64,i64,i64,i32,i64,i32,i32,i64,i32,i64,i64,i32,i64,i64,i32,i64,i32,i64,i64,i64,i32,i32,i64,i64,i32,i64,i64,i64,i64,i64,…,i32,i32,i32,i32,i64,i32,i32,i32,i32,i32,i32,i64,i32,i32,i64,i32,i32,i32,i32,i32,i32,i32,i64,i32,i32,i32,i64,i32,i32,i32,i32,i32,i64,i64,i32,i32,i32
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,0,0,2,0,19,0,0,0,0,0,85,0,145,0,0,0,76,0,0,0,299,0,288,0,0,0,0,0,0,0,415,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,95,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""Buenos Aires""","""25 de Mayo""","""Agustín Mosconi""",6854010,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""Buenos Aires""","""25 de Mayo""","""Del Valle""",6854020,0,0,1,0,0,0,0,0,0,0,181,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""Buenos Aires""","""25 de Mayo""","""Ernestina""",6854030,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""Buenos Aires""","""25 de Mayo""","""Gobernador Ugarte""",6854040,0,0,0,0,0,0,0,106,0,0,3,0,0,56,0,7,3,0,0,0,4,0,1,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [66]:
"""
columna 0 ---> provincia
columna 1 ---> partido
columna 2 ---> localidad
columna 3 ---> id_indec
"""

# tomo las columnas de "Otros" en adelante
columns = rangos_localidades_2.columns[4:]
list_rangos_localidades = []
for i in range(rangos_localidades_2.shape[0]):
    for j,column in enumerate(columns):
        if rangos_localidades_2[i,4+j] != 0:
            row = {}
            row['provincia'] = rangos_localidades_2[i,0]
            row['partido'] = rangos_localidades_2[i,1]
            row['localidad'] = rangos_localidades_2[i,2]
            row['id_indec'] = rangos_localidades_2[i,3]
            row['velocidad_mbps'] = -10 if column == 'Otros' else float(column[:column.find(' ')].replace(',','.'))
            row['accesos'] = rangos_localidades_2[i,4+j]
            list_rangos_localidades.append(row)
 
rangos_localidades = pl.DataFrame(list_rangos_localidades)
print(rangos_localidades.shape)
del rangos_localidades_2
# Verificando cuantas velocidades distintas existen
print(f"{rangos_localidades['velocidad_mbps'].unique().count()} distintas velocidades")
rangos_localidades.head(10)

(18599, 6)
50 distintas velocidades


provincia,partido,localidad,id_indec,velocidad_mbps,accesos
str,str,str,i64,f64,i64
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,0.5,2
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,0.75,19
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,3.0,85
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,3.5,145
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,6.0,76
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,8.0,299
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,10.0,288
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,15.0,415
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,20.0,1797


***
# Tablas `partidos` y `localidades`

In [70]:
localidades_de_rangos = rangos_localidades['provincia', 'partido', 'localidad', 'id_indec'].unique().to_dicts()
localidades_de_accesos = accesos_localidades['provincia', 'partido', 'localidad', 'id_indec'].to_dicts()

print(f'Hay  {len(localidades_de_rangos)} localidades distintas en "rangos_localidades"')
print(f'Hay  {len(localidades_de_accesos)} localidades distintas en "accesos_localidades"')

Hay  2983 localidades distintas en "rangos_localidades"
Hay  3002 localidades distintas en "accesos_localidades"


En el for de abajo recorro la lista de diccionario mas pequeña y diccionario que no se encuentre dentro de la lista mas grande, se lo anexo

In [71]:
for diccionario in localidades_de_rangos:
    if not diccionario in localidades_de_accesos:
        localidades_de_accesos.append(diccionario)

print(f'En Total hay {len(localidades_de_accesos)} localidades distintas entre estas dos tablas')

En Total hay 3021 localidades distintas entre estas dos tablas


Voy a Normalizar las Tablas `localidades` y `partidos` SOLO CON ESTAS localidades que son las que voy a utilizar

In [72]:
# Estas son las Localidades distintas que pueden haber solo trabajando con estas dos tablas
localidades = pl.DataFrame(localidades_de_accesos)

# Estos son los partidos distintos que pueden haber solo trabajando con estas dos tablas
partidos = localidades['provincia', 'partido'].unique()

# Creo el indice de localidad
localidades = localidades.with_row_index(name='id_localidad', offset=1)

# Creo el indice de paritdo
partidos = partidos.with_row_index(name='id_partido', offset=1)

In [75]:
localidades.head(1)

id_localidad,provincia,partido,localidad,id_indec
u32,str,str,str,i64
1,"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100


In [76]:
partidos.head(1)

id_partido,provincia,partido
u32,str,str
1,"""Chaco""","""12 de Octubre"""


Verificando que solo hay 24 provincias en partidos y localidades

In [77]:
localidades['provincia'].unique().shape

(24,)

In [78]:
partidos['provincia'].unique().shape

(24,)

Sustituyendo `provincia` por `id_provincia` dentro de `partidos`

In [79]:
partidos = partidos.join(
    provincias['id_provincia','provincia'], on='provincia', how='left', coalesce=True
)

partidos = partidos.drop('provincia')

partidos.head(1)

id_partido,partido,id_provincia
u32,str,i64
1,"""12 de Octubre""",22


Verificando que los Join se comportaron bien y no quedaron nulos

In [81]:
for column in partidos.columns:
    print(f'Columna --> {column}')
    print(f'Cantidad de Nulos --> {partidos[column].is_null().sum()}')
    print('*'*50)

Columna --> id_partido
Cantidad de Nulos --> 0
**************************************************
Columna --> partido
Cantidad de Nulos --> 0
**************************************************
Columna --> id_provincia
Cantidad de Nulos --> 0
**************************************************


In [82]:
# Creo el id_provincia necesario para el sigiuente Join
localidades = localidades.join(
    provincias['id_provincia', 'provincia'], on='provincia', how='left', coalesce=True
)

localidades = localidades.join(
    partidos, on=['id_provincia', 'partido'], how='left', coalesce=True
)

localidades.head(1)

id_localidad,provincia,partido,localidad,id_indec,id_provincia,id_partido
u32,str,str,str,i64,i64,u32
1,"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,6,387


Anexo un id_localidad a las tablas `accesos_localidades` y `accesos_localidades`

In [84]:
columns = ['id_localidad', 'provincia', 'partido', 'localidad']
join_columns = ['provincia', 'partido', 'localidad']

rangos_localidades = rangos_localidades.join(
    localidades[columns], on=join_columns, how='left', coalesce=True
)

accesos_localidades = accesos_localidades.join(
    localidades[columns], on=join_columns, how='left', coalesce=True
)

In [85]:
rangos_localidades.head(5)

provincia,partido,localidad,id_indec,velocidad_mbps,accesos,id_localidad
str,str,str,i64,f64,i64,u32
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,0.5,2,1
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,0.75,19,1
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,3.0,85,1
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,3.5,145,1
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,6.0,76,1


In [87]:
accesos_localidades.head(5)

provincia,partido,localidad,id_indec,adsl,cablemodem,dial_up,fibra,otros,satelital,winmax,wireless,total,poblacion,_3g,latitud,_4g,longitud,id_localidad
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,f64,bool,f64,u32
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,755,4600,0,2,0,742,0,727,6826,23408,true,-35.433939,true,-60.173121,1
"""Buenos Aires""","""25 de Mayo""","""Del Valle""",6854020,181,0,0,0,10,1,0,0,192,899,true,-35.897345,false,-60.731609,2
"""Buenos Aires""","""25 de Mayo""","""Gobernador Ugarte""",6854040,0,0,0,0,0,0,0,181,181,547,false,-35.164397,false,-60.081358,3
"""Buenos Aires""","""25 de Mayo""","""Norberto de la Riestra""",6854060,0,782,0,496,16,0,0,185,1479,4524,true,-35.272779,true,-59.771933,4
"""Buenos Aires""","""25 de Mayo""","""Lucas Monteverde""",6854050,0,0,0,0,0,0,0,6,6,63,false,-35.483827,false,-59.988065,5


Ya puedo Borrarle las columnas redundantes a `localidades`

In [92]:
localidades = localidades.drop(['provincia', 'partido', 'id_provincia'])
localidades.head(1)

id_localidad,localidad,id_indec,id_partido
u32,str,i64,u32
1,"""25 de Mayo""",6854100,387


Y debo anexarle información a Localidades

In [93]:
columns = ['id_localidad', 'poblacion', 'longitud', 'latitud']
localidades = localidades.join(
    accesos_localidades[columns], on='id_localidad', how='left', coalesce=True
)

localidades.head(1)

id_localidad,localidad,id_indec,id_partido,poblacion,longitud,latitud
u32,str,i64,u32,i64,f64,f64
1,"""25 de Mayo""",6854100,387,23408,-60.173121,-35.433939


Pasare a 0 los campos que no tuvieron coincidencias en el Join comentado de los  87 Nulos en  `accesos_localidades` y en la tabla `localidades` 

In [ ]:
# mask = accesos_localidades['poblacion'].is_null()
# accesos_localidades = accesos_localidades.filter(~mask)

columns = localidades.columns[4:]
for column in columns:
    localidades = localidades.with_columns(
        pl.col(column).fill_null(0).alias(column)
    )

    accesos_localidades = accesos_localidades.with_columns(
        pl.col(column).fill_null(0).alias(column)
    )

**Guardaré estas tablas tal y como están para el EDA y Dashboard, y luego le borraré las columnas redundantes para guardar en `SQL`**

In [ ]:
accesos_localidades.write_parquet('tablas/accesos_localidades_eda.parquet')

rangos_localidades.write_parquet('tablas/rangos_localidades_eda.parquet')

In [ ]:
accesos_localidades.head(1)

In [207]:
rangos_localidades = rangos_localidades.drop(['provincia', 'partido', 'localidad'])

accesos_localidades = accesos_localidades.drop(['provincia', 'partido', 'localidad', 'longitud', 'latitud', 'poblacion'])

In [95]:
a_mano = pl.DataFrame({
    'id_localidad': 699,
    'id_indec': 2000010,
    'poblacion': 3121707,
    '_3g': True,
    '_4g': True,
    'longitud': -58.445876,
    'latitud': -34.614442
})

In [106]:
columns = ['poblacion', 'longitud', 'latitud']
for column in columns:
    localidades = localidades.with_columns(
        pl.when(pl.col('id_indec') == a_mano['id_indec'][0])
          .then(pl.lit(a_mano[column][0]))
          .otherwise(pl.col(column))
          .alias(column)
    )

columns = ['_3g', '_4g']
for column in columns:
    accesos_localidades = accesos_localidades.with_columns(
        pl.when(pl.col('id_indec') == a_mano['id_indec'][0])
          .then(pl.lit(a_mano[column][0]))
          .otherwise(pl.col(column))
          .alias(column)
    )    

In [108]:
accesos_localidades.head(1)

provincia,partido,localidad,id_indec,adsl,cablemodem,dial_up,fibra,otros,satelital,winmax,wireless,total,poblacion,_3g,latitud,_4g,longitud,id_localidad
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,f64,bool,f64,u32
"""Buenos Aires""","""25 de Mayo""","""25 de Mayo""",6854100,755,4600,0,2,0,742,0,727,6826,23408,true,-35.433939,true,-60.173121,1


***
Guardado de tablas de `Internet`

In [127]:
provincias.write_parquet('tablas/provincias.parquet')

partidos.write_parquet('tablas/partidos.parquet')

localidades.write_parquet('tablas/localidades.parquet')

internet_pais.write_parquet('tablas/internet_pais.parquet')

internet_provincia.write_parquet('tablas/internet_provincia.parquet')

accesos_x_tecnologia.write_parquet('tablas/accesos_x_tecnologia.parquet')

velocidad_sin_rangos.write_parquet('tablas/velocidad_sin_rangos.parquet')

accesos_localidades.write_parquet('tablas/accesos_localidades.parquet')

rangos_localidades.write_parquet('tablas/rangos_localidades.parquet')

***
#### **Toca alimentar nuestra Base de Datos Mysql**

Creando la Conexión

In [112]:
engine = engine_sqlalchemy()

Uno a uno se pasa cada Dataframe a un df de Pandas y luego se vacia en su respectiva tabla `mysql`.  Esta sintáxis es genérica sirve para cualquier `motor de Sql` que tenga compatibilidad con `SqlAlchemy` 

In [126]:
df_pandas = provincias.to_pandas()
df_pandas.to_sql(name="provincias", con=engine, index=False, if_exists="append")

df_pandas = partidos.to_pandas()
df_pandas.to_sql(name="partidos", con=engine, index=False, if_exists="append")

df_pandas = localidades.to_pandas()
df_pandas.to_sql(name="localidades", con=engine, index=False, if_exists="append")

df_pandas = internet_pais.to_pandas()
df_pandas.to_sql(name="internet_pais", con=engine, index=False, if_exists="append")

df_pandas = internet_provincia.to_pandas()
df_pandas.to_sql(name="internet_provincia", con=engine, index=False, if_exists="append")

df_pandas = accesos_x_tecnologia.to_pandas()
df_pandas.to_sql(name="accesos_x_tecnologia", con=engine, index=False, if_exists="append")

df_pandas = velocidad_sin_rangos.to_pandas()
df_pandas.to_sql(name="velocidad_sin_rangos", con=engine, index=False, if_exists="append")

df_pandas = accesos_localidades.to_pandas()
df_pandas.to_sql(name="accesos_localidades", con=engine, index=False, if_exists="append")

df_pandas = rangos_localidades.to_pandas()
df_pandas.to_sql(name="rangos_localidades", con=engine, index=False, if_exists="append")

18599

In [128]:
accesos_localidades = accesos_localidades.join(
    localidades, on='id_localidad', how='left', coalesce=True 
)

accesos_localidades = accesos_localidades.join(
    partidos, on='id_partido', how='left', coalesce=True
)

accesos_localidades = accesos_localidades.join(
    provincias['id_provincia', 'provincia'], on='id_provincia', how='left', coalesce=True
)

accesos_localidades = accesos_localidades.with_columns(
    (pl.col('total') / pl.col('poblacion') * 100).round(2).alias('penetracion')
)


In [132]:
accesos_localidades.write_parquet('tablas/accesos_localidades_eda.parquet')